In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly
import numpy as np
from scipy import stats
import shap


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score, f1_score, log_loss, matthews_corrcoef
%matplotlib inline

pd.set_option('display.max_columns', None)

In [ ]:
test = pd.read_csv('test_data.csv', index_col='case_id')
train = pd.read_csv('train_data.csv', index_col='case_id')

train['train_or_test'] = 'train'
test['train_or_test'] = 'test'

In [ ]:
train.head()

Based on the data type we have to encode the variables of 'Stay' and 'Age' from string into integer using LabelEncoder (These variables are called ordinal data, we want to turn it into integer but still keeping it's magnitude)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for a in ['Stay', 'Age']:
    le.fit(np.sort(train[a].unique()))
    train[a] = le.transform(train[a])
    
le.fit(np.sort(test['Age'].unique()))
test['Age'] = le.transform(test['Age'])

df = pd.concat([train,test])

df_train = df[df['train_or_test'] == 'train']
df_test = df[df['train_or_test'] == 'test'].drop(columns=['Stay'])

In [ ]:
plt.figure(figsize=(20,6))
sns.countplot(df_train['Stay'], order=np.sort(df_train['Stay'].unique()).tolist())

In [ ]:
plt.figure(figsize=(20,6))
sns.countplot(df_train['Age'], order=np.sort(df_train['Age'].unique()).tolist())

Also the variable 'Severity of Illness' comes in the form of ordinal data to, we should encode them into integers

In [ ]:
df['Severity of Illness'] = df['Severity of Illness'].map({'Minor': 1, 'Moderate': 2, 'Extreme': 3})

In [ ]:
sns.heatmap(df.select_dtypes(exclude='object').corr(method='spearman'))

In [ ]:
sns.heatmap(df_train.isnull())

In [ ]:
df_train.isnull().any()

Bed Grade and City_Code_Patient have null values, if we want to build our model using these features, we have to deal them first by imputing with the estimator (median or mean). But for now, let's explore our data first.

**EDA**

In [ ]:
sns.countplot(data=df, x='Hospital_type_code', order=df['Hospital_type_code'].value_counts().index)

Hospital type 'a' has the most admissions than any other hospital type, hence the hospital type 'a' is more likely to run out of available beds or rooms. The ideal distribution of available beds or rooms would be uniform distribution, because we want to avoid a situation where a hospital has to turn away a critically-ill patient because there's no available room for the patient.

In [ ]:
fig1 = px.sunburst(df, path=['Hospital_region_code','Hospital_type_code'])
fig1.update_layout(title='Hospital region case load composition',title_x=0.5)
fig1.show()

**Note:** <br>

1. In region 'X', the hospital type 'a' has the most case than the other.
1. Region 'Y' type 'a' and 'b' hospital has a well-balanced case load, but type 'd', 'f', and 'g' are not utilized well.
1. Same case with region 'Z', type 'c' hospital has the most case then type 'a' and 'd' hospital.

In [ ]:
fig2 = px.histogram(df ,x='Available Extra Rooms in Hospital', color='Hospital_region_code', marginal='box', opacity=0.7)
fig2.update_layout(title='Available extra rooms histogram', title_x=0.5)
fig2.show()

As we can see from the above histogram, majority of hospitals have extra rooms in the size of 2,3 or 4 rooms. The median extra rooms is 3

In [ ]:
room_dist = df.groupby(['Hospital_region_code', 'Hospital_code'])['Available Extra Rooms in Hospital'].agg(pd.Series.median).to_frame().reset_index()

In [ ]:
fig3 = px.pie(room_dist, values='Available Extra Rooms in Hospital', names='Hospital_region_code', hole=0.4)
fig3.update_layout(title='Available rooms per hospital mode in each region composition',title_x=0.5)
fig3.update_traces(textinfo='percent+label')

In [ ]:
room_dist

From the histogram and pie chart above, we can see the number of extra rooms available for each region are almost equally distributed.<br>

This means that in each region when there is a patient admitted to a hospital the number of the available rooms is usually the same as the other hospitals. (we use the mode of available room for each hospital)

In [ ]:
fig4 = px.pie(values=df['Department'].value_counts().values, names=df['Department'].value_counts().index, hole=0.4)
fig4.update_layout(title='Department case composition',title_x=0.5)
fig4.update_traces(textinfo='percent+label')

78.3% of our case are being handled by gynecology department.

In [ ]:
fig5 = px.pie(values=df['Severity of Illness'].value_counts().values, names=df['Severity of Illness'].value_counts().index, hole=0.4)
fig5.update_layout(title='Severity case composition',title_x=0.5)
fig5.update_traces(textinfo='percent+label')

Half (55.2%) of our case is moderately-severe illness. Let's explore it a little further.

In [ ]:
fig6 = px.sunburst(df_train, path=['Age','Severity of Illness'])
fig6.update_layout(title='Age & Severity composition',title_x=0.5)
fig6.show()

The composition of severity for each class of Age is just the same as any other class. Moderate severity is the most common followed by Minor and then Extreme severity.

In [ ]:
fig7 = px.sunburst(df_train, path=['Stay','Severity of Illness'])
fig7.update_layout(title='Stay & Severity composition',title_x=0.5)
fig7.show()

The longer the patient stay, the more likely the patient has severe illness. Although in each class of Stay the Moderate severity is the most common, notice that Extreme severity case is starting to rise at 5.0 stay class (51-60 days)

In [ ]:
df['patient_hosp_same_code'] = df['City_Code_Patient'] == df['City_Code_Hospital']

In [ ]:
fig8 = px.pie(values=df['patient_hosp_same_code'].value_counts().values, names=df['patient_hosp_same_code'].value_counts().index, hole=0.4)
fig8.update_layout(title='Patient Hospital Same City Composition',title_x=0.5)
fig8.update_traces(textinfo='percent+label')

In [ ]:
fig9 = px.sunburst(df, path=['patient_hosp_same_code','Severity of Illness'])
fig9.update_layout(title='Patient Hospital Same City & Severity composition',title_x=0.5)
fig9.update_traces(textinfo='percent root+label')

Notice that only 7% of case, the hospital and the patient has the same city code. It means that almost every patient has to travel outside of their city to get treatment at the hospital. Ideally, we want the patient treated quickly by admitting them to a hospital in their own city, or we could make sure the patient treated immediately if they have to travel outside of their city.<br>

This is why hospital room management is so important because we couldn't afford to put the patient in any more danger. By making sure that every time a patient is admitted to a hospital there's always an extra room available, we could save more lives.

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(data=df_train, x='City_Code_Hospital', y='Stay')

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(data=df_train, x='City_Code_Hospital', y='Stay')

Hospitals in the city code of 10 has the highest means of Stay duration than any other city code.

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(data=df_train, x='Hospital_type_code', y='Stay')

In [ ]:
plt.figure(figsize=(8,4))
sns.boxplot(data=df_train, x='Hospital_type_code', y='Stay')

Hospital type g has the highest mean of stay than any other type.

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(data=df_train, x='Ward_Type', y='Stay')

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(data=df_train, x='Ward_Facility_Code', y='Stay')

In [ ]:
df['risk_factor'] = df['Age'] * df['Severity of Illness']

In [ ]:
sns.barplot(data=df, x='risk_factor', y='Stay')

risk_factor is just a combination of Age and Severity of Illness, to picture how likely a patient has to stay longer in hospital (intuitively, we know that the older and the more ill the patient are, the longer they're likely to stay)

In [ ]:
# import packages for WOE IV
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
tmp = df[df['train_or_test']=='train'].copy()
# a code to count how many visits the patient has
tmp['patient_hosp'] = tmp['patientid'].astype(str) + '_' + tmp['Hospital_code'].astype(str)
counter = tmp['patient_hosp'].value_counts()
tmp['patient_revisit_hosp'] = tmp['patient_hosp'].apply(lambda x: counter[x])

tmp.drop(columns=['patientid', 'train_or_test', 'patient_hosp', 'Age', 'Severity of Illness'], axis=1, inplace=True)

In [ ]:
final_iv, IV = data_vars(tmp.drop(columns=['Stay']), tmp['Stay'])

In [ ]:
final_iv

In [ ]:
top_feature = IV.sort_values('IV', ascending=False)
top_feature_name = top_feature.head(10)['VAR_NAME'].values.tolist()
top_feature_name.append('Stay')

In [ ]:
sns.barplot(data=top_feature, x='IV', y='VAR_NAME')

In [ ]:
top_feature_name

Now we found our top ten features that have the most information value, we can use those only to build our model faster without sacrificing predicting metrics score.<br>

Because Hospital_code is categorical and has many unique values. We probably should drop it.

In [ ]:
top_feature_name = top_feature.head(9)['VAR_NAME'].values.tolist()
top_feature_name.append('Stay')
top_feature_name

In [ ]:
df['patient_hosp'] = df['patientid'].astype(str) + '_' + df['Hospital_code'].astype(str)
counter = df['patient_hosp'].value_counts()
df['patient_revisit_hosp'] = df['patient_hosp'].apply(lambda x: counter[x])

In [ ]:
dffe = df[df['train_or_test'] == 'train'].drop(columns=[
                        'train_or_test'], axis=1)[top_feature_name].copy()

In [ ]:
dffe.info()

In [ ]:
dffe_ohe = pd.get_dummies(dffe, drop_first=True)

Now that our data is ready, let's build our model using CatBoost

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
cb = CatBoostClassifier()
data = dffe_ohe.drop(columns=['Stay'], axis=1)
target = dffe_ohe['Stay']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data, target, random_state=101, test_size=0.3)

In [ ]:
cb.fit(X_train, y_train)

In [ ]:
print(classification_report(y_train, cb.predict(X_train)))

In [ ]:
print(classification_report(y_val, cb.predict(X_val)))

The accuracy of train and validation data are not that different (0.43 and 0.40). This means that our model is JUST-FIT (not overfit and underfit). Let's see the score with cross-validation technique.

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=cb,
                        X=X_train,
                        y=y_train,
                        cv=4,
                        n_jobs=-1,
                        scoring = 'accuracy')

print('Cross validation scores: {}'.format(scores))

plt.title('Cross validation scores')
plt.scatter(np.arange(len(scores)), scores)
plt.axhline(y=np.mean(scores), color='g') # Mean value of cross validation scores
plt.show()

**Model Explanation**

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(cb)

In [ ]:
X_train.shape[1]

In [ ]:
shap_values = explainer.shap_values(Pool(X_train, y_train))

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar")

Let's try implementing PCA with Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier

pipe_cb = Pipeline([('std_scl', StandardScaler()), 
                    ('pca', PCA(n_components=X_train.shape[1])),
                    ('catboost', CatBoostClassifier(random_state=101))])

pipe_cb.fit(X_train, y_train)

print('Test Accuracy: {}'.format(pipe_cb.score(X_val, y_val)))

In [ ]:
print(classification_report(y_train, pipe_cb.predict(X_train)))

In [ ]:
print(classification_report(y_val, pipe_cb.predict(X_val)))

In [ ]:
from sklearn.model_selection import GridSearchCV

search_cb = GridSearchCV(
                estimator=CatBoostClassifier(random_state=101),
                param_grid = {
                    'learning_rate' : [0.05, 0.1],
                    'depth' : [6, 8],
                    'l2_leaf_reg' : [3, 4],
                    'loss_function' : ['MultiClass']},
                scoring = 'accuracy',
                cv = 3,
                n_jobs = -1)

In [ ]:
search_cb.fit(X_train, y_train)

In [ ]:
search_cb.best_score_

In [ ]:
print(classification_report(y_val, search_cb.predict(X_val)))

In [ ]:
dffe = df.drop(columns=['train_or_test'], axis=1)[top_feature_name].copy()

In [ ]:
dffe.info()

In [ ]:
dffe_ohe = pd.get_dummies(dffe, drop_first=True)

In [ ]:
X_train = dffe_ohe[dffe_ohe['Stay'].notnull()].drop(columns=['Stay']).copy()
y_train = dffe_ohe[dffe_ohe['Stay'].notnull()]['Stay'].copy()
X_test = dffe_ohe[dffe_ohe['Stay'].isnull()].drop(columns=['Stay']).copy()

In [ ]:
X_test

In [ ]:
cb.fit(X_train, y_train)

In [ ]:
print(classification_report(y_train, cb.predict(X_train)))

In [ ]:
result = cb.predict(X_test).copy()
np.transpose(result)
result

In [ ]:
df_submission = pd.DataFrame({
    'case_id': X_test.index,
})


In [ ]:
df_submission['Stay'] = result

In [ ]:

df_submission['Stay'] = df_submission['Stay'].map({
    0.0 : '0-10',
    1.0 : '11-20',
    2.0 : '21-30',
    3.0 : '31-40',
    4.0 : '41-50',
    5.0 : '51-60',
    6.0 : '61-70',
    7.0 : '71-80',
    8.0 : '81-90',
    9.0 : '91-100',
    10.0 : 'More than 100 Days'
})

df_submission.to_csv(path_or_buf='hakim_submission.csv', index=False)